# Neural Network Assigment 2

## Tensorflow

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [9]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [10]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [11]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
y = tf.matmul(x,W) + b

In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [15]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

## Keras

In [19]:
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, Deconv2D, AveragePooling2D
from keras.layers import Input, add, concatenate
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split
from keras import backend as K

In [1]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Using TensorFlow backend.


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
X_train = np.expand_dims(X_train,3)
X_test = np.expand_dims(X_test,3)

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [7]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [8]:
def norm_input(x):
    return (x-mean_px)/std_px

In [9]:
(X_train.shape, y_train.shape, X_test.shape, y_test.shape,)

((60000, 28, 28, 1), (60000, 10), (10000, 28, 28, 1), (10000, 10))

In [36]:
def get_model(input_shape=(28, 28, 1), units=10, lr=0.001, drop_rate=0.25):
    
    inputs = Input(input_shape)
    x = Flatten()(inputs)
    x = Dense(units)(x)
    x = Dense(units)(x)
    x = Dropout(drop_rate)(x)
    x = Activation('softmax')(x) 
    
    model = Model(inputs=inputs, outputs=x)


    return model

In [37]:
model = get_model()

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                7850      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
activation_5 (Activation)    (None, 10)                0         
Total params: 7,960
Trainable params: 7,960
Non-trainable params: 0
_________________________________________________________________


In [39]:
opt = Adadelta(lr=0.01, decay=1e-5)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
%time model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data = (X_test, y_test),shuffle = True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 9s - loss: 2.7420 - acc: 0.8200 - val_loss: 2.6757 - val_acc: 0.8204
Epoch 2/10
60000/60000 [==============================] - 9s - loss: 2.6841 - acc: 0.8230 - val_loss: 2.6060 - val_acc: 0.8235
Epoch 3/10
60000/60000 [==============================] - 9s - loss: 2.6290 - acc: 0.8258 - val_loss: 2.5432 - val_acc: 0.8271
Epoch 4/10
60000/60000 [==============================] - 9s - loss: 2.5605 - acc: 0.8294 - val_loss: 2.4800 - val_acc: 0.8309
Epoch 5/10
60000/60000 [==============================] - 8s - loss: 2.4890 - acc: 0.8331 - val_loss: 2.3948 - val_acc: 0.8356
Epoch 6/10
60000/60000 [==============================] - 7s - loss: 2.4028 - acc: 0.8379 - val_loss: 2.3394 - val_acc: 0.8397
Epoch 7/10
60000/60000 [==============================] - 7s - loss: 2.3178 - acc: 0.8426 - val_loss: 2.2947 - val_acc: 0.8423
Epoch 8/10
60000/60000 [==============================] - 9s 